# U.S. STOCK MARKET RETURN

In [2]:
from pandas_datareader import DataReader as pdr

mkt = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]/100
mkt = mkt['Mkt-RF'] + mkt.RF
mkt.index.name = "year"
mkt.name = "market"
mkt.index = mkt.index.astype(str)
mkt.tail()

year
2018   -0.0512
2019    0.3043
2020    0.2411
2021    0.2360
2022   -0.2017
Name: market, dtype: float64

# RETURNS FROM YAHOO FINANCE

In [3]:
import yfinance as yf

ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]
price = price.resample("Y").last()
ser = price.pct_change()[:-1].dropna()
ser.index = ser.index.map(lambda x: x.year)
ser.index.name = "year"
ser.name = ticker
ser.tail()

[*********************100%***********************]  1 of 1 completed


year
2018   -0.097474
2019    0.152737
2020   -0.259521
2021    0.462413
2022    0.584623
Name: cvx, dtype: float64

# YAHOO RETURN OR MARKET

In [4]:
import numpy as np
import pandas as pd

flag = int(input("input 0 for market or 1 for yahoo return"))
ret = ser if flag else mkt
name = f"{ticker.upper()} Return" if flag else "Market Return"
ccret = np.log(1+ret)
data = pd.concat((ret, ccret), axis=1)
data.columns = ["Return", "CC Return"]

# TABLE

In [6]:
data.describe().iloc[1:]

,Return,CC Return
mean,0.142744,0.112567
std,0.232506,0.208578
min,-0.316027,-0.379838
25%,-0.017183,-0.017509
50%,0.116748,0.110420
75%,0.289680,0.254375
max,0.764967,0.568132


# FIGURE 1

In [11]:
import plotly.graph_objects as go

trace = go.Scatter(
    x=data["Return"],
    y=data["CC Return"], 
    mode="markers", 
    text=data.index.to_list(),
    hovertemplate="%{text}<br>Return = %{x:.1%}<br>Cont. Comp. Ret = %{y:.1%}"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Return",
    yaxis_title="Continuously Compounded Return",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",  
)
fig.show()

# FIGURE 2

In [13]:
import plotly.figure_factory as ff

fig = ff.create_distplot(
    [data["Return"], data["CC Return"]], 
    group_labels=["Return", "CC Return"], 
    show_rug=False, 
    show_hist=False
)

fig.update_traces(hovertemplate="")
fig.update_layout(
    xaxis_tickformat="0%",
    xaxis_title="",
    yaxis_tickvals=[],

    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
    ),
    template="plotly_white"
)
fig.show()